In [1]:
import os 
import time
import json
import nltk
import string
import pickle
import datetime

In [2]:
import numpy as np
import pandas as pd
import xml.etree.ElementTree as xml

In [3]:
from moviepy.editor import *
from tqdm.notebook import tqdm
from collections import Counter

from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

strptime = datetime.datetime.strptime

In [4]:
# shots_selection_path = '../facerec_segment/segs/segments_add2_keep2_cut0.7.pickle'
facerec_path = '../facerec_segment/facerec_query_characters.csv'
facerec_selected_path = '../facerec_segment/chosen_segments_0.7.csv'
facerec_all_path = '../facerec_segment/facerec_query_characters_coref.csv'
mentions_path = '../coref/summaries_query_mentions.pickle'
subtitles_path = '../transcripts/shot-aligned_transcripts.csv'
descriptions_path = '../transcripts/shot-aligned_captions.csv'
captions_path = '../captions/deep_captions.csv'
mastershot_path = '../transcripts/mastershot_timing.csv'

In [5]:
file2id = {  '5531550228324592939.mp4': '175',
             '5534228999422914578.mp4': '176',
             '5539381671692122744.mp4': '177',
             '5542003749222140011.mp4': '178',
             '5544574287152993687.mp4': '179',
             '5544620672795594434.mp4': '180',
             '5547193787702629969.mp4': '181',
             '5549784941472309008.mp4': '182',
             '5552368364300855101.mp4': '183',
             '5555325449284154780.mp4': '184',
             '5555360238519252381.mp4': '185'}

eps = [  '5531550228324592939.mp4',
         '5534228999422914578.mp4',
         '5539381671692122744.mp4',
         '5542003749222140011.mp4',
         '5544574287152993687.mp4',
         '5544620672795594434.mp4',
         '5547193787702629969.mp4',
         '5549784941472309008.mp4',
         '5552368364300855101.mp4',
         '5555325449284154780.mp4',
         '5555360238519252381.mp4', ]

In [6]:
mastershot_df = pd.read_csv('../transcripts/mastershot_timing.csv')
mastershot_df['start'] = mastershot_df['start'].apply(lambda t: t[:8] + '.' + t[9:])
mastershot_df['end'] = mastershot_df['end'].apply(lambda t: t[:8] + '.' + t[9:])
mastershot_df['sid'] = mastershot_df['file'].apply(lambda x: file2id[x.replace('xml', 'mp4')] + '-') + mastershot_df['shot'].astype(str)
mastershot_df.head()

,Unnamed: 0,file,shot,start_s,end_s,start,end,sid
0,0,5531550228324592939.xml,1,0.00,0.12,00:00:00.00,00:00:00.12,175-1
1,1,5531550228324592939.xml,2,0.13,0.24,00:00:00.13,00:00:00.24,175-2
2,2,5531550228324592939.xml,3,1.00,1.18,00:00:01.00,00:00:01.18,175-3
3,3,5531550228324592939.xml,4,1.19,2.05,00:00:01.19,00:00:02.05,175-4
4,4,5531550228324592939.xml,5,2.06,24.13,00:00:02.06,00:00:24.13,175-5


In [7]:
df_facerec = pd.read_csv(facerec_path)
df_facerec['sid'] = df_facerec['file'].apply(lambda x: file2id[x] + '-') + df_facerec['shot_id'].astype(str)
df_facerec = df_facerec[df_facerec['confidence'] > 0.5]
df_facerec.head()

,Unnamed: 0,file,shot_id,start,end,start_s,end_s,duration,character,confidence,sid
0,0,5555360238519252381.mp4,18,00:00:59:04,00:01:03:15,59.04,63.15,4.11,Stacey Branning,0.647979,185-18
1,1,5555360238519252381.mp4,19,00:01:03:16,00:01:05:06,63.16,65.06,1.90,Stacey Branning,0.718986,185-19
5,5,5555360238519252381.mp4,25,00:01:48:02,00:02:01:01,108.02,121.01,12.99,Stacey Branning,0.542448,185-25
6,6,5555360238519252381.mp4,25,00:01:48:02,00:02:01:01,108.02,121.01,12.99,Ryan Malloy,0.589787,185-25
8,8,5555360238519252381.mp4,25,00:01:48:02,00:02:01:01,108.02,121.01,12.99,Ryan Malloy,0.573998,185-25


In [131]:
sorted(df_facerec.duration.unique(), reverse=True)[:10]

[55.98000000000002,
 48.960000000000036,
 46.1500000000001,
 45.03999999999997,
 43.98999999999978,
 42.01000000000022,
 41.039999999999964,
 40.95000000000027,
 39.98999999999978,
 39.08000000000004]

In [8]:
df_facerec_selected = pd.read_csv(facerec_selected_path)
df_facerec_selected = df_facerec_selected[df_facerec_selected['avg_confidence'] > 0.5]
df_facerec_selected['sid'] = df_facerec_selected['file'].apply(lambda x: file2id[x] + '-') + df_facerec_selected['shot_id'].astype(str)
df_facerec_selected.head()

,Unnamed: 0,file,shot_id,character,avg_confidence,sid
0,0,5555360238519252381.mp4,19,Stacey Branning,0.718986,185-19
1,1,5555360238519252381.mp4,36,Janine Malloy,0.833666,185-36
2,2,5555360238519252381.mp4,38,Janine Malloy,0.877764,185-38
3,3,5555360238519252381.mp4,42,Ryan Malloy,0.823699,185-42
4,4,5555360238519252381.mp4,46,Ryan Malloy,0.837216,185-46


In [9]:
df_facerec_all = pd.read_csv(facerec_all_path)
df_facerec_all = df_facerec_all[df_facerec_all['confidence'] > 0.5]
df_facerec_all['sid'] = df_facerec_all['file'].apply(lambda x: file2id[x] + '-') + df_facerec_all['shot_id'].astype(str)
df_facerec_all.head()

,Unnamed: 0,file,shot_id,start,end,start_s,end_s,duration,character,confidence,sid
57,57,5555360238519252381.mp4,36,00:02:42:05,00:02:56:06,162.05,176.06,14.01,Janine Malloy,0.533121,185-36
58,58,5555360238519252381.mp4,36,00:02:42:05,00:02:56:06,162.05,176.06,14.01,Janine Malloy,0.537654,185-36
62,62,5555360238519252381.mp4,36,00:02:42:05,00:02:56:06,162.05,176.06,14.01,Janine Malloy,0.504680,185-36
66,66,5555360238519252381.mp4,36,00:02:42:05,00:02:56:06,162.05,176.06,14.01,Janine Malloy,0.519436,185-36
67,67,5555360238519252381.mp4,36,00:02:42:05,00:02:56:06,162.05,176.06,14.01,Janine Malloy,0.544696,185-36


In [10]:
first_names = lambda chars: ' '.join(set([char.split()[0] for char in chars]))

df_chars_queries = pd.DataFrame(df_facerec.groupby('sid')['character'].apply(first_names)).rename(columns={'character':'chars_query'})
df_chars_queries.reset_index(level=0, inplace=True)

df_chars_selected = pd.DataFrame(df_facerec_selected.groupby('sid')['character'].apply(first_names)).rename(columns={'character': 'selected_character'})
df_chars_selected.reset_index(level=0, inplace=True)


df_chars_all = pd.DataFrame(df_facerec_all.groupby('sid')['character'].apply(first_names)).rename(columns={'character':'chars_all'})
df_chars_all.reset_index(level=0, inplace=True)

df_chars_all

,sid,chars_all
0,175-1002,Billy
1,175-1012,Billy
2,175-1015,Billy
3,175-1027,Ryan
4,175-1031,Ryan
5,175-1033,Ryan
6,175-1037,Ryan
7,175-1041,Billy
8,175-1045,Ryan
9,175-1046,Ryan


In [203]:
df_chars_all.head(5)

,sid,chars_all
0,175-1002,Billy
1,175-1012,Billy
2,175-1015,Billy
3,175-1027,Ryan
4,175-1031,Ryan


In [204]:
df_chars_selected.head()

,sid,selected_character
0,175-1027,Ryan
1,175-1028,Ryan
2,175-1029,Ryan
3,175-1031,Ryan
4,175-1033,Ryan


In [205]:
df_chars_queries.head(5)

,sid,chars_query
0,175-10,Ryan
1,175-1002,Ryan
2,175-1004,Ryan
3,175-1005,Ryan
4,175-1006,Ryan


In [11]:
mentions = pickle.load(open(mentions_path, 'rb'))
print('support', mentions['5531550228324592939.mp4']['support'][:2], '..')
print('marked', mentions['5531550228324592939.mp4']['marked'][:2], '..')
print('characters', mentions['5531550228324592939.mp4']['characters'][:2], '..')
print('queries', mentions['5531550228324592939.mp4']['queries'][:2], '..')

support ['Kat persuades Stacey and Jean to come to R&R with her and Kim, leaving Charlie and Alfie to look after Lily', 'Lily won’t settle so Pat takes her for a walk'] ..
marked ['#Kat persuades @Stacey and #Jean to come to ~R&R with #Kat and #Kim , leaving #Charlie and #Alfie to look after ~Lily', '@Janine accuses @Ryan of wanting to go to ~R&R because ~R&R knew @Stacey would be there'] ..
characters [['Kat', 'Stacey', 'Jean', 'Kat', 'Kim', 'Charlie', 'Alfie'], ['Janine', 'Ryan', 'Stacey']] ..
queries [['Stacey'], ['Janine', 'Ryan', 'Stacey']] ..


In [12]:
subtitles = pd.read_csv(subtitles_path)
subtitles['sid'] = subtitles['filename'].apply(lambda x: file2id[x.replace('xml', 'mp4')] + '-') + subtitles['shot_id'].astype(str)
subtitles.head()

,Unnamed: 0,filename,shot_id,begin,end,transcript,sid
0,0,5531550228324592939.xml,1,0.00,0.12,NaN,175-1
1,1,5531550228324592939.xml,2,0.13,0.24,NaN,175-2
2,2,5531550228324592939.xml,3,1.00,1.18,NaN,175-3
3,3,5531550228324592939.xml,4,1.19,2.05,NaN,175-4
4,4,5531550228324592939.xml,5,2.06,24.13,NaN,175-5


In [13]:
captions = pd.read_csv(captions_path)
captions['sid'] = captions['episode'].apply(lambda x: file2id[x] + '-') + captions['shot'].astype(str)
captions.head()

,Unnamed: 0,episode,shot,caption,sid
0,0,5531550228324592939.mp4,1,a man is riding a skateboard and doing tricks,175-1
1,1,5531550228324592939.mp4,2,a man is skateboarding down a ramp,175-2
2,2,5531550228324592939.mp4,3,a man is riding a skateboard and doing tricks,175-3
3,3,5531550228324592939.mp4,4,a car is driving on a track and then crashes i...,175-4
4,4,5531550228324592939.mp4,5,a bird is perched on a tree branch,175-5


In [209]:
len(captions)

21212

In [14]:
descriptions = pd.read_csv(descriptions_path).rename(columns={'caption': 'description'})
descriptions['sid'] = descriptions['filename'].apply(lambda x: file2id[x.replace('xml', 'mp4')] + '-') + descriptions['shot_id'].astype(str)
descriptions.head()

,Unnamed: 0,filename,shot_id,begin,end,description,sid
0,0,5531550228324592939.xml,1,0.00,0.12,NaN,175-1
1,1,5531550228324592939.xml,2,0.13,0.24,NaN,175-2
2,2,5531550228324592939.xml,3,1.00,1.18,NaN,175-3
3,3,5531550228324592939.xml,4,1.19,2.05,NaN,175-4
4,4,5531550228324592939.xml,5,2.06,24.13,NaN,175-5


In [15]:
df = subtitles[['sid', 'filename', 'begin', 'end', 'transcript']].merge(
                descriptions[['sid', 'description']], on = 'sid', how='outer').merge(
                captions[['sid', 'caption']], on = ['sid'], how='outer').merge(
                df_chars_queries[['sid', 'chars_query']], on = ['sid'], how='outer').merge(
                df_chars_selected[['sid', 'selected_character']], on = ['sid'], how='outer').merge(
                df_chars_all[['sid', 'chars_all']], on = ['sid'], how='outer').fillna('')

In [17]:
df.to_csv('transcrip_vsum_2021.csv')

In [16]:
df[(df['description'] != '')]

,sid,filename,begin,end,transcript,description,caption,chars_query,selected_character,chars_all
119,175-120,5531550228324592939.xml,313.11,315.23,,DOOR SLAMS,a man is sitting on a bed and talking,Ryan,,
237,175-238,5531550228324592939.xml,679.15,688.00,,BABY CRIES,a man is dancing in a kitchen,,,
246,175-247,5531550228324592939.xml,746.20,750.16,"OK, we've got a limited target tonight, girls...",THEY LAUGH,a woman is dancing in front of a crowd,Ryan,,
262,175-263,5531550228324592939.xml,777.11,779.17,,THEY LAUGH,a man is dancing with a woman,Ryan,Ryan,
345,175-346,5531550228324592939.xml,990.16,991.16,,CRASHING AND GLASS BREAKING,a man is looking at something and then looks away,,,
375,175-376,5531550228324592939.xml,1030.03,1034.24,,GIRLS LAUGH,a woman is dancing in front of a group of people,Ryan,,
388,175-389,5531550228324592939.xml,1088.06,1097.04,"You want some of this, do ya? Show me what yo...",SHE LAUGHS,a man is dancing with a woman in a dress,Ryan,,
424,175-425,5531550228324592939.xml,1191.06,1196.11,"I've got to go and pick her up on the bus, an...",PAT LAUGHS,a man is talking to a woman and she is smiling,Ryan,,
433,175-434,5531550228324592939.xml,1214.10,1218.02,"Oh, Lord.",BABY STARTS TO CRY,a man is looking at something and then he look...,,,
481,175-482,5531550228324592939.xml,1331.19,1333.19,,WOMAN ON TV SPEAKS AND BABY CRIES,a woman is sitting at a table with a man,,,


In [214]:
df[df['transcript'] != '']['transcript'][:30]

6     Getting all dolled up, going somewhere nice to...
9                          Yeah, curry night at Dad's. 
10    Wicked.  If you like yesterday's chicken with ...
12    In your condition it's nice to have  someone c...
13    My condition?  I'm pregnant, not paralysed. Ev...
15    I know. But look,  you've got to take it easy....
16    All right, "I ain't got  to take it anything."...
18    What it says it is, special night  at R&R's. H...
20    Yeah? What's happy about it?  Happy hour all n...
23       and I'm going to put it out there.  Yes, I am.
25    Sounds great.  So you coming tonight, then? Yo...
29                    Hi. I've not got an appointment. 
30                     No, it's all right, no worries. 
33    I'll be with you in a minute.  Just take a sea...
35               How about you, Janice?  R&R's tonight?
37    Happy Monday.  It'll be Miserable Monday if sh...
38    Oh, Kat, I didn't see you there  underneath al...
39    Not if she's going.  Oh, don't worry about

In [215]:
df['content'] = df['transcript'] + ' ' + ' ' + df['description'] + ' ' +  df['chars_query']
df['content']

0                                                         
1                                                         
2                                                         
3                                                         
4                                                         
                               ...                        
21207    Download the new release now  from the Doctor ...
21208                                                     
21209                                                     
21210                                                 Ryan
21211                                                     
Name: content, Length: 21212, dtype: object

In [216]:
ps = PorterStemmer()
ls = LancasterStemmer()

In [217]:
ps.stem('kills')

'kill'

In [218]:
do_stem = True

vectorizer = TfidfVectorizer(min_df=3, stop_words='english')

corpus = [' '.join(ps.stem(w) for w in word_tokenize(s) if w.lower() not in vectorizer.get_stop_words()) if do_stem else s for s in df['content'].values]
corpus = [s.translate(str.maketrans('', '', string.punctuation)).replace('  ', ' ').lower() for s in corpus]


vectorizer.fit(corpus)
corpus_tf_idf = vectorizer.transform(corpus)

In [219]:
corpus[94]

'excus re go tonight re go good time tv new york knickerbock correct ryan stacey'

In [220]:
word_count = Counter([w for s in corpus for w in s.split()])
print(len(word_count))

4886


In [221]:
word_count

Counter({'get': 124,
         'doll': 2,
         'go': 828,
         'nice': 100,
         'tonight': 59,
         'ryan': 9089,
         'yeah': 1133,
         'curri': 10,
         'night': 138,
         'dad': 197,
         's': 3709,
         'wick': 6,
         'like': 662,
         'yesterday': 25,
         'chicken': 6,
         'tin': 8,
         'sauc': 1,
         'chuck': 9,
         'condit': 6,
         'cook': 25,
         'innit': 57,
         'm': 1050,
         'pregnant': 10,
         'paralys': 1,
         'carri': 18,
         'oi': 151,
         'babi': 103,
         're': 949,
         'talk': 228,
         'know': 1120,
         'look': 634,
         've': 888,
         'got': 805,
         'easi': 23,
         'ai': 342,
         'nt': 3103,
         'right': 1080,
         'come': 636,
         'say': 357,
         'special': 28,
         'r': 22,
         'happi': 94,
         'monday': 9,
         'hour': 48,
         'thomp': 1,
         'own': 2,
         

In [222]:
corpus_tf_idf.shape

(21212, 1913)

In [223]:
print(len(vectorizer.get_stop_words()))
', '.join(sorted(vectorizer.get_stop_words()))

318


'a, about, above, across, after, afterwards, again, against, all, almost, alone, along, already, also, although, always, am, among, amongst, amoungst, amount, an, and, another, any, anyhow, anyone, anything, anyway, anywhere, are, around, as, at, back, be, became, because, become, becomes, becoming, been, before, beforehand, behind, being, below, beside, besides, between, beyond, bill, both, bottom, but, by, call, can, cannot, cant, co, con, could, couldnt, cry, de, describe, detail, do, done, down, due, during, each, eg, eight, either, eleven, else, elsewhere, empty, enough, etc, even, ever, every, everyone, everything, everywhere, except, few, fifteen, fifty, fill, find, fire, first, five, for, former, formerly, forty, found, four, from, front, full, further, get, give, go, had, has, hasnt, have, he, hence, her, here, hereafter, hereby, herein, hereupon, hers, herself, him, himself, his, how, however, hundred, i, ie, if, in, inc, indeed, interest, into, is, it, its, itself, keep, las

In [224]:
tfidf_weights = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

In [225]:
tfidf_weights['rape']

9.170610011044296

In [226]:
tfidf_weights['man']

6.134055742970049

In [227]:
tfidf_weights['zealand']

9.352931567838251

In [228]:
summaries_raw = [s for ep in eps for s in mentions[ep]['marked']]
summaries_raw = [' '.join(w[1:] if w[0] in ['@', '#', '~'] else w for w in s.split(' ')) for s in summaries_raw]
summaries_raw[:10]

['Kat persuades Stacey and Jean to come to R&R with Kat and Kim , leaving Charlie and Alfie to look after Lily',
 'Janine accuses Ryan of wanting to go to R&R because R&R knew Stacey would be there',
 'Ryan denies it and leaves , Janine stays in the club',
 'Julie explains that Janine was moved to another home when pregnant , their baby was taken away a week after Janine gave birth',
 'Stacey realises Billy ’s upset Stacey mum , Janine and Kat get drawn into their argument before Alfie steps in',
 'Giving Lily to Ryan , Pat follows Bianca into the club , worried',
 'Billy then turns on Stacey , who hits Billy',
 'Janine pulls out a chunk of Stacey ’s hair',
 'Kat and Janine hit each other',
 'Kim , Kat , Stacey , Pat and Janine are shut in a cell together']

In [229]:
summaries = [' '.join(ps.stem(w) for w in word_tokenize(s) if w.lower() not in vectorizer.get_stop_words()) if do_stem else s for s in summaries_raw]
summaries = [s.translate(str.maketrans('', '', string.punctuation)).replace('  ', ' ') for s in summaries]

summaries[:10]

['kat persuad stacey jean come R R kat kim leav charli alfi look lili',
 'janin accus ryan want R R R R knew stacey',
 'ryan deni leav janin stay club',
 'juli explain janin move home pregnant babi taken away week janin gave birth',
 'stacey realis billi ’ s upset stacey mum janin kat drawn argument alfi step',
 'give lili ryan pat follow bianca club worri',
 'billi turn stacey hit billi',
 'janin pull chunk stacey ’ s hair',
 'kat janin hit',
 'kim kat stacey pat janin shut cell']

In [230]:
def compute_similarity(content, summary_line, method):
    if method[0] == 'tfidf':
        weights = method[1]
        return sum(weights[w] if w in weights else 0 for w in set(content.split()) if w in summary_line.split())
    if method[0] == 'tfidf_log':
        weights = method[1]
        dn = np.log(len(content.split() + summary_line.split()) + 1) if len(content.strip().split()) > 0 else 1
        return sum(weights[w] if w in weights else 0 for w in set(content.split()) if w in summary_line.split())/dn
    if method[0] == 'tfidf_log_2':
        weights = method[1]
        n = sum(1 if w in weights else 0 for w in set(content.split()) if w in summary_line.split())
        dn = np.log(len(content.split() + summary_line.split()) - n + 1) 
        dn = dn if dn > 0 else 1
        return sum(weights[w] if w in weights else 0 for w in set(content.split()) if w in summary_line.split())/dn
    if method[0] == 'tfidf_log_3':
        weights = method[1]
        n = sum(1 if w in weights else 0 for w in set(content.split()) if w in summary_line.split())
        dn = np.log10(len(content.split() + summary_line.split()) - n + 1) 
        dn = dn if dn > 0 else 1
        return sum(weights[w] if w in weights else 0 for w in set(content.split()) if w in summary_line.split())/dn
    if method[0] == 'tfidf_sqrt':
        weights = method[1]
        n = sum(1 if w in weights else 0 for w in set(content.split()) if w in summary_line.split())
        dn = np.sqrt(len(content.split() + summary_line.split()) - n + 1) 
        dn = dn if dn > 0 else 1
        return sum(weights[w] if w in weights else 0 for w in set(content.split()) if w in summary_line.split())/dn
    elif method[0] == 'count':
        return len([w for w in set(content.split()) if w in summary_line.split()])
    elif method[0] == 'count_2':
        dn = np.log(len(content.split() + summary_line.split()) + 1) 
        dn = dn if dn > 0 else 1
        return len([w for w in set(content.split()) if w in summary_line.split()]) / dn

In [231]:
compute_similarity('No more excus . a woman is talk to a man and he is smile TV : ryan stacey',
                   'kat persuad stacey and jean to come to R & R with kat and kim , leav charli and alfi to look after lili',
                    method=('tfidf', tfidf_weights)), compute_similarity('No more excus . a woman is talk to a man and he is smile TV : ryan stacey',
                   'kat persuad stacey and jean to come to R & R with kat and kim , leav charli and alfi to look after lili',
                    method=('count',))

(3.337750494112952, 3)

In [232]:
tfidf_weights['stacey']

3.337750494112952

In [233]:
%%time
similarity_matrix_count = np.zeros((len(df), len(summaries)))
for i, content in tqdm(enumerate(corpus), total=len(corpus)):
    for j, s in enumerate(summaries):
        similarity_matrix_count[i, j] = compute_similarity(content, s, method=('count',))


CPU times: user 29.4 s, sys: 117 ms, total: 29.5 s
Wall time: 29.7 s


In [234]:
unrolled_sm_count = [(similarity_matrix_count[i, j], i, j)  for i in range(21212) for j in range(388)]
best_matches_count = sorted(unrolled_sm_count, key=lambda x: -x[0])

In [235]:
best_matches_count[0]

(10.0, 3614, 36)

In [236]:
for v, i, j in best_matches_count[:3]:
    print(df['content'].iloc[i])
    print(summaries_raw[j])
    print(v)
    print(list(w for w in set(summaries[j].split()) if w in corpus[i].split()))

Janine let you out then?  She doesn't own me. Good. I'm going down the Town Hall to  get Lily's birth certificate changed. Why today?  Cos I'm working after. Come on, you, me and Lily. It'll be fun.  All right?   Ryan Stacey
Stacey tells Ryan Stacey ’s going to change Lily ’s birth certificate - if Ryan ’s serious about Ryan daughter , Ryan ’ll come
10.0
['chang', 'stacey', 'ryan', 'go', 'certif', 'lili', 's', 'll', 'birth', 'come']
Can you just let it go?  I can't. It's over, babe. It's history.  I know it was her. How do you know?  How can you possibly know that? If it wasn't Bradley and it  wasn't you, it has to be her. What, is that it?  Why else was he running away from the police that night? He must've been protecting Stacey.  If you go round shouting about this, you'll ruin an innocent girl's life. Why can't you just admit it?  What do you want me to admit?  What, Archie Mitchell was a nasty  piece of work? Yeah, course he was. And loads of people had  a beef with him, Lauren, b

In [237]:
# similarity_matrix_tfidf = similarity_matrix

In [238]:
%%time
similarity_matrix_tfidf = np.zeros((len(df), len(summaries)))
for i, content in tqdm(enumerate(corpus), total=len(corpus)):
    for j, s in enumerate(summaries):
        similarity_matrix_tfidf[i, j] = compute_similarity(content, s, method=('tfidf_log_2', tfidf_weights))


CPU times: user 1min 37s, sys: 387 ms, total: 1min 37s
Wall time: 1min 38s


In [239]:
np.max(similarity_matrix_tfidf)

15.975222291317356

In [240]:
similarity_matrix_tfidf.shape

(21212, 388)

In [241]:
unrolled_sm = [(similarity_matrix_tfidf[i, j], i, j) for i in range(21212) for j in range(388)]
best_matches = sorted(unrolled_sm, key=lambda x: -x[0])

In [242]:
best_matches[:10]

[(15.975222291317356, 6983, 64),
 (13.910569594982768, 940, 12),
 (11.96736503631256, 3614, 36),
 (11.222736383738184, 19791, 299),
 (11.222736383738184, 19791, 373),
 (10.818253525815793, 6575, 59),
 (10.700604880163784, 1771, 26),
 (10.502068075280782, 8165, 299),
 (10.502068075280782, 8165, 373),
 (10.140584904299763, 14552, 204)]

In [243]:
for v, i, j in best_matches[:20]:
    print(i, df['content'].iloc[i])
    print(j, summaries_raw[j])
    print()

6983 I said "stop talking"!  You killed Archie. And you as good  as pushed Bradley off the roof too.   
64 Lauren accuses Stacey of not just killing Archie , but as good as pushing Bradley off the roof

940 had a baby with somebody  else's husband. Yeah? I've never been  engaged to a dirty old rapist.   Stacey
12 When Janine taunts Stacey , Stacey blurts out ‘ I ’ve never been engaged to a dirty old rapist ’

3614 Janine let you out then?  She doesn't own me. Good. I'm going down the Town Hall to  get Lily's birth certificate changed. Why today?  Cos I'm working after. Come on, you, me and Lily. It'll be fun.  All right?   Ryan Stacey
36 Stacey tells Ryan Stacey ’s going to change Lily ’s birth certificate - if Ryan ’s serious about Ryan daughter , Ryan ’ll come

19791 You didn't kill Archie?    Stacey
299 Stacey tells them she did kill Archie

19791 You didn't kill Archie?    Stacey
373 Stacey tells them she did kill Archie

6575 # Nothing more, nothing less  Love is the best... # You

# Making the cut

In [249]:
best_matches[:5]

[(15.975222291317356, 6983, 64),
 (13.910569594982768, 940, 12),
 (11.96736503631256, 3614, 36),
 (11.222736383738184, 19791, 299),
 (11.222736383738184, 19791, 373)]

In [250]:
score = {}
for (v, s, l) in best_matches:
    if s not in score : score[s] = []
    score[s].append(1)
for s in score:
    score[s] = sum(score[s])

In [251]:
reses = sorted(score.items(), key=lambda x: -x[1])

In [252]:
votes_selection = {c:[] for c in ['Janine', 'Ryan', 'Stacey']}
votes_durations = {c:[] for c in ['Janine', 'Ryan', 'Stacey']}

for char in votes_selection:
    print(char.upper() + ' =====>')
    counter = 1
    for s in reses:
        if char in df['content'].iloc[s[0]]:
            print(df['content'].iloc[s[0]])
            votes_selection[char].append(df['sid'].iloc[s[0]])
            votes_durations[char].append(df['end'].iloc[s[0]] - df['begin'].iloc[s[0]])
        if len(votes_selection[char]) == 20: 
            break

JANINE =====>
Janine let you out then?  She doesn't own me. Good. I'm going down the Town Hall to  get Lily's birth certificate changed. Why today?  Cos I'm working after. Come on, you, me and Lily. It'll be fun.  All right?   Ryan Stacey
# Nothing more, nothing less  Love is the best... # You all right?  I know. Archie Mitchell's killer. I've  worked it out. Your precious Stacey.   Ryan Janine Stacey
Any time you need me, you know  where I am. Please, Janine. Please?  You've dropped your bombshell,  Pat, why don't you just fly off to New Zealand and enjoy  yourself with your real family. I don't know why I bother  caring about you.   Ryan Janine
I know you want nice little neat  answers. Life's not like that. Don't patronise me.  I'm not a kid any more. Then you should know you can't go  jumping to conclusions and calling it the truth cos that's  what you want to hear. Swear on my life  that it wasn't Stacey. I swear...  on your life...  that Stacey never killed  Archie Mitchell.   Ja

In [256]:
top_selection = {c:[] for c in ['Janine', 'Ryan', 'Stacey']}
top_durations = {c:[] for c in ['Janine', 'Ryan', 'Stacey']}
top_times     = {c:[] for c in ['Janine', 'Ryan', 'Stacey']}

for char in top_selection:
    print(char.upper() + ' =====>')
    counter = 1
    seen = set()
    for (v, s, l) in best_matches:
        if s not in seen and char in df['content'].iloc[s] + df['chars_query'].iloc[s]:
            sentence = df['transcript'].iloc[s].strip()
            print(sentence)
            
            top_selection[char].append(df['sid'].iloc[s])
            top_durations[char].append(round(df['end'].iloc[s] - df['begin'].iloc[s], 2))
            top_times[char].append((df['begin'].iloc[s], df['end'].iloc[s]))

            seen.add(s)
            while sentence and sentence[-1] not in ['.', '?', '!', "'"]:
                s += 1
                sentence = df['transcript'].iloc[s].strip()
                if sentence == '':
                    break
                print(' ++++ ', sentence)

                top_selection[char].append(df['sid'].iloc[s])
                top_durations[char].append(round(df['end'].iloc[s] - df['begin'].iloc[s], 2))
                top_times[char].append((df['begin'].iloc[s], df['end'].iloc[s]))
                seen.add(s)
                counter +=1
                
            counter += 1
        
        if len(seen) >= 20: 
            break

JANINE =====>
Janine let you out then?  She doesn't own me. Good. I'm going down the Town Hall to  get Lily's birth certificate changed. Why today?  Cos I'm working after. Come on, you, me and Lily. It'll be fun.  All right?
# Nothing more, nothing less  Love is the best... # You all right?  I know. Archie Mitchell's killer. I've  worked it out. Your precious Stacey.
Any time you need me, you know  where I am. Please, Janine. Please?  You've dropped your bombshell,  Pat, why don't you just fly off to New Zealand and enjoy  yourself with your real family. I don't know why I bother  caring about you.
I know you want nice little neat  answers. Life's not like that. Don't patronise me.  I'm not a kid any more. Then you should know you can't go  jumping to conclusions and calling it the truth cos that's  what you want to hear. Swear on my life  that it wasn't Stacey. I swear...  on your life...  that Stacey never killed  Archie Mitchell.
Look, whoever finds  the best Christmas present wins.

In [297]:
for char in top_selection:
    print(char.upper())
    for run in range(1, 5):
        print('Run', run, ':', ' '.join(sorted(top_selection[char][:5*run])))

JANINE
Run 1 : 176-1200 177-2024 178-51 182-14 182-1681
Run 2 : 176-1200 176-1785 176-2060 177-2024 178-51 181-116 182-14 182-1494 182-1681 183-1411
Run 3 : 175-2247 176-1200 176-1785 176-2060 177-2024 178-51 181-116 181-1544 182-1108 182-14 182-1494 182-1681 182-676 183-1411 185-992
Run 4 : 175-2247 176-1200 176-1785 176-2060 176-928 177-2024 178-51 181-116 181-1544 182-1108 182-14 182-1494 182-1681 182-676 183-1397 183-1411 183-1883 184-39 185-943 185-992
RYAN
Run 1 : 175-1772 176-1200 177-2024 178-1549 182-1681
Run 2 : 175-1772 176-1092 176-1200 177-2024 178-1549 179-1741 182-1203 182-1681 183-1411 185-438
Run 3 : 175-1772 176-1092 176-1200 176-2060 177-2024 178-1549 179-1741 179-940 181-1084 181-116 182-1203 182-1494 182-1681 183-1411 185-438
Run 4 : 175-1772 175-2247 176-1092 176-1200 176-1242 176-2060 177-2024 178-1549 179-1741 179-940 181-1084 181-116 181-1250 181-1544 182-1203 182-1279 182-1494 182-1681 183-1411 185-438
STACEY
Run 1 : 175-941 176-1200 177-2024 178-1549 185-316


In [298]:
for char in top_durations:
    print(char.upper())
    for run in range(1, 5):
        print('Run', run, ':', sum(top_durations[char][:5*run]))

JANINE
Run 1 : 70.37
Run 2 : 110.13000000000001
Run 3 : 128.97
Run 4 : 174.03
RYAN
Run 1 : 64.69
Run 2 : 121.7
Run 3 : 166.69
Run 4 : 198.14000000000001
STACEY
Run 1 : 52.66
Run 2 : 83.72999999999999
Run 3 : 126.74
Run 4 : 158.09999999999997


In [259]:
for char in votes_durations:
    print(char.upper())
    for run in range(1, 5):
        print('Run', run, ':', sum(votes_durations[char][:5*run]))

JANINE
Run 1 : 70.37000000000016
Run 2 : 110.1300000000018
Run 3 : 128.97000000000287
Run 4 : 174.03000000000267
RYAN
Run 1 : 64.69000000000051
Run 2 : 121.70000000000073
Run 3 : 166.69000000000148
Run 4 : 198.1400000000022
STACEY
Run 1 : 52.660000000000196
Run 2 : 93.0399999999998
Run 3 : 139.8
Run 4 : 162.10000000000065


In [291]:
top_times

{'Janine': [(3904.24, 3917.08),
  (7070.01, 7088.24),
  (5542.12, 5558.14),
  (238.06, 258.23),
  (37.09, 40.2),
  (4743.24, 4747.24),
  (5791.23, 5796.1),
  (4854.11, 4867.08),
  (347.02, 348.08),
  (6681.23, 6698.09),
  (6703.17, 6709.13),
  (5130.19, 5132.1),
  (2986.18, 2989.0),
  (2101.16, 2106.21),
  (3491.02, 3494.12),
  (2861.11, 2867.01),
  (6411.17, 6417.17),
  (4668.06, 4688.21),
  (2883.13, 2886.05),
  (138.01, 148.1)],
 'Ryan': [(3904.24, 3917.08),
  (7070.01, 7088.24),
  (5326.22, 5328.05),
  (5167.23, 5183.0),
  (5542.12, 5558.14),
  (3545.24, 3565.09),
  (1333.04, 1345.19),
  (5856.08, 5864.03),
  (3807.08, 3820.14),
  (4743.24, 4747.24),
  (3746.02, 3749.06),
  (4854.11, 4867.08),
  (3070.13, 3081.19),
  (347.02, 348.08),
  (6681.23, 6698.09),
  (4212.14, 4228.04),
  (6703.17, 6709.13),
  (4064.21, 4067.01),
  (4068.14, 4073.02),
  (5130.19, 5132.1)],
 'Stacey': [(2688.13, 2691.01),
  (3904.24, 3917.08),
  (883.21, 886.15),
  (7070.01, 7088.24),
  (5167.23, 5183.0),
  

In [261]:
pickle.dump(top_selection, open('selected_shots_final.pickle', 'wb'))
pickle.dump(top_times, open('selected_shots_boundaries_final.pickle', 'wb'))

# Generate the Videos!

In [281]:
! pip install -q moviepy

In [299]:
shot_boundaries = {}
for character in top_selection:
    for shot in top_selection[character]:
        if shot not in shot_boundaries:
            shot_boundaries[shot] = mastershot_df[mastershot_df.sid == shot][['start', 'end']].values[0]

In [300]:
set([k.split('-')[0] for k in shot_boundaries])

{'175', '176', '177', '178', '179', '181', '182', '183', '184', '185'}

In [301]:
shot_boundaries.keys()

dict_keys(['176-1200', '177-2024', '182-1681', '178-51', '182-14', '183-1411', '176-1785', '182-1494', '181-116', '176-2060', '175-2247', '181-1544', '185-992', '182-676', '182-1108', '176-928', '183-1883', '183-1397', '185-943', '184-39', '175-1772', '178-1549', '176-1092', '185-438', '179-1741', '182-1203', '181-1084', '179-940', '181-1250', '176-1242', '182-1279', '175-941', '185-316', '185-981', '184-2212', '184-2213', '182-1509', '182-1188', '177-1793', '182-57', '184-1785'])

In [ ]:
shot185_1736

In [284]:
3763.13 * 25

94078.25

In [285]:
os.listdir('../../eastenders_episodes')

['5531550228324592939.mp4',
 '5534228999422914578.mp4',
 '5542003749222140011.mp4',
 '5555325449284154780.mp4',
 '5547193787702629969.mp4',
 '5544620672795594434.mp4',
 '5555360238519252381.mp4',
 '5552368364300855101.mp4',
 '5549784941472309008.mp4',
 '5544574287152993687.mp4',
 '5539381671692122744.mp4']

In [286]:
"""%%time
frames = {}
# Read until video is completed
for filename in tqdm(os.listdir('../../eastenders_episodes'), total=11):
    print('Processing', filename)
    vid = cv2.VideoCapture('../../eastenders_episodes/' + filename)
    vid_id = file2id[filename]
    current_frame = 0
    
    for sid in sorted([sid for sid in shot_boundaries if sid.startswith(vid_id)]):
        start_frame = int(shot_boundaries[shot][0] * 25)
        end_frame = int(shot_boundaries[shot][1] * 25)
        frames[sid] = []
        
        while(cap.isOpened()):
            ret, frame = cap.read()
            current_frame += 1
            if ret == True and current_frame >= start_frame and current_frame <= end_frame:
                frames[sid].append(frame)
            if ret == True and current_frame > end_frame:
                print(f'frames[{sid}]:', len(frames[sid]))
                break

    vid.release()"""
print('Using OpenCV is too slow')

Using OpenCV is too slow


In [287]:
df[df.sid == '175-1772'][['begin', 'end']].values[0]

array([5326.22, 5328.05])

In [303]:
%%time
frames = {}
durs = {}
# Read until video is completed
for filename in tqdm(os.listdir('../../eastenders_episodes'), total=11):
    print('Processing', filename)
    video = VideoFileClip('../../eastenders_episodes/' + filename)
    vid_id = file2id[filename]
    current_frame = 0
    
    for sid in sorted([sid for sid in shot_boundaries if sid.startswith(vid_id)]):
        try:
            start, end = shot_boundaries[sid]
            frames[sid] = video.subclip(start, end)
            durs[sid] = (strptime(end, '%H:%M:%S.%f') - strptime(start, '%H:%M:%S.%f')).total_seconds()
        except Exception as e:
            print('EXCEPTION', str(e))
            print(sid, start, end)

Processing 5531550228324592939.mp4
Processing 5534228999422914578.mp4
Processing 5542003749222140011.mp4
Processing 5555325449284154780.mp4
Processing 5547193787702629969.mp4
Processing 5544620672795594434.mp4
Processing 5555360238519252381.mp4
Processing 5552368364300855101.mp4
Processing 5549784941472309008.mp4
Processing 5544574287152993687.mp4
Processing 5539381671692122744.mp4

CPU times: user 130 ms, sys: 4.15 s, total: 4.28 s
Wall time: 18.2 s


In [304]:
for char in top_selection:
    print(char.upper())
    print('Generating videos for', char, ':')
    for run in range(1, 5):
        t = time.time()
        print('   Run', run, '..', end=' ')
        shots = sorted(top_selection[char][:5*run])
        result = concatenate_videoclips([frames[sid] for sid in shots]) 
        result.write_videofile(f"videos/MeMAD_{run}_{char}.mp4",fps=25, logger=None)
        print(f'Done! ({(time.time() - t):.2f} s)')

JANINE
Generating videos for Janine :
   Run 1 .. Done! (20.18 s)
   Run 2 .. Done! (37.67 s)
   Run 3 .. Done! (45.12 s)
   Run 4 .. Done! (58.39 s)
RYAN
Generating videos for Ryan :
   Run 1 .. Done! (20.10 s)
   Run 2 .. Done! (37.25 s)
   Run 3 .. Done! (53.85 s)
   Run 4 .. Done! (68.36 s)
STACEY
Generating videos for Stacey :
   Run 1 .. Done! (20.21 s)
   Run 2 .. Done! (28.77 s)
   Run 3 .. Done! (44.57 s)
   Run 4 .. Done! (59.39 s)


In [351]:
results = {'Janine': {1:[('175_1', 5.3), ('175_2', 5.3)],
                      2:[('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3)],
                      3:[('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3), ('175_4', 5.3)],
                      4:[('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3), ('175_4', 5.3), ('175_5', 5.3)],},
           'Ryan':   {1:[('175_1', 5.3), ('175_2', 5.3)],
                      2:[('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3)],
                      3:[('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3), ('175_4', 5.3)],
                      4:[('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3), ('175_4', 5.3), ('175_5', 5.3)],},
           'Stacey': {1:[('175_1', 5.3), ('175_2', 5.3)],
                      2:[('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3)],
                      3:[('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3), ('175_4', 5.3)],
                      4:[('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3), ('175_4', 5.3), ('175_5', 5.3)],}}

In [354]:
results

{'Janine': {1: [('175_1', 5.3), ('175_2', 5.3)],
  2: [('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3)],
  3: [('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3), ('175_4', 5.3)],
  4: [('175_1', 5.3),
   ('175_2', 5.3),
   ('175_3', 5.3),
   ('175_4', 5.3),
   ('175_5', 5.3)]},
 'Ryan': {1: [('175_1', 5.3), ('175_2', 5.3)],
  2: [('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3)],
  3: [('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3), ('175_4', 5.3)],
  4: [('175_1', 5.3),
   ('175_2', 5.3),
   ('175_3', 5.3),
   ('175_4', 5.3),
   ('175_5', 5.3)]},
 'Stacey': {1: [('175_1', 5.3), ('175_2', 5.3)],
  2: [('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3)],
  3: [('175_1', 5.3), ('175_2', 5.3), ('175_3', 5.3), ('175_4', 5.3)],
  4: [('175_1', 5.3),
   ('175_2', 5.3),
   ('175_3', 5.3),
   ('175_4', 5.3),
   ('175_5', 5.3)]}}

In [367]:
results = {}
for char in top_selection:
    results[char] = {}
    for run in range(1, 5):
        shots = sorted(top_selection[char][:5*run])
        results[char][run] = [(shot.replace('-', '_'), durs[shot]) for shot in shots] # [('100-1', 10.5) for shot in shots] #

In [368]:
results

{'Janine': {1: [('176_1200', 12.84),
   ('177_2024', 18.23),
   ('178_51', 20.17),
   ('182_14', 3.11),
   ('182_1681', 16.02)],
  2: [('176_1200', 12.84),
   ('176_1785', 4.87),
   ('176_2060', 16.86),
   ('177_2024', 18.23),
   ('178_51', 20.17),
   ('181_116', 1.06),
   ('182_14', 3.11),
   ('182_1494', 12.97),
   ('182_1681', 16.02),
   ('183_1411', 4.0)],
  3: [('175_2247', 5.96),
   ('176_1200', 12.84),
   ('176_1785', 4.87),
   ('176_2060', 16.86),
   ('177_2024', 18.23),
   ('178_51', 20.17),
   ('181_116', 1.06),
   ('181_1544', 1.91),
   ('182_1108', 3.1),
   ('182_14', 3.11),
   ('182_1494', 12.97),
   ('182_1681', 16.02),
   ('182_676', 5.05),
   ('183_1411', 4.0),
   ('185_992', 2.82)],
  4: [('175_2247', 5.96),
   ('176_1200', 12.84),
   ('176_1785', 4.87),
   ('176_2060', 16.86),
   ('176_928', 5.9),
   ('177_2024', 18.23),
   ('178_51', 20.17),
   ('181_116', 1.06),
   ('181_1544', 1.91),
   ('182_1108', 3.1),
   ('182_14', 3.11),
   ('182_1494', 12.97),
   ('182_1681',

In [376]:
def generate_results(results):
    team_name = 'MeMAD'
    desc = "The output"
    query_characters = ['Janine', 'Ryan', 'Stacey']
    shots_per_run = {1: 5, 2: 10, 3:15, 4:20}

    for run in [1, 2, 3, 4]:
        with open(f'xml/{team_name}_run_{run}.xml', 'wt') as file:
            file.write('<!DOCTYPE videoSummarizationResults SYSTEM "https://www-nlpir.nist.gov/projects/tv2020/dtds/videoSummarizationResults.dtd">\n')
            file.write('<videoSummarizationResults>\n')
            file.write(f'\t<videoSummarizationRunResult pid="{team_name}" priority="{run}" desc="Run {run}">\n')
            for character in query_characters:
                n_shots = shots_per_run[run]
                duration = sum([c[1] for c in results[character][run]])
                file.write(f'\t\t<videoSummarizationTopicResult  target="{character}" numShots="{shots_per_run[run]}" summTime="{round(duration, 1)}">\n')
                for i, shot_id in enumerate(results[character][run]):
                    file.write(f'\t\t\t<item seqNum="{i+1}" shotId="shot{shot_id[0]}"/>\n')
                file.write(f'\t\t</videoSummarizationTopicResult>\n')
            file.write('\t</videoSummarizationRunResult>\n')
            file.write('</videoSummarizationResults>\n')

In [377]:
generate_results(results)